<a href="https://colab.research.google.com/github/slayerzeroa/Railway_Accident_BERT/blob/main/Railway_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setting
+ Installing
+ Call Library
+ Set Cuda
+ Google Api Key

In [1]:
!pip install transformers
!pip install nltk
!pip install pytorch-crf
!pip install konlpy
!pip install collections

!pip install gspread
!pip install oauth2client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.2 MB/s 
     |████████████████████████████████| 7.6 MB 41.9 MB/s 
     |████████████████████████████████| 182 kB 56.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 435 kB/s 
     |████████████████████████████████| 465 kB 67.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
Looking in indexes: https://pypi.org/simple, https://us-python.pk

In [2]:
# 라이브러리
import torch
from torch import nn, optim, tensor
from torch.utils.data import DataLoader, Dataset,TensorDataset, random_split
import torch.nn.functional as F
from torchcrf import CRF

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, mean_squared_error

from transformers import BertTokenizer, BertForMaskedLM, AdamW, get_linear_schedule_with_warmup, BertConfig, BertForTokenClassification, BertModel, BertTokenizerFast

import json
import glob
import re
import os
import ast
import html

import re
import requests

import zipfile
import shutil

import nltk

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

from konlpy.tag import Okt
from collections import Counter

from google.colab import files

In [3]:
#punkt 다운로드
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
#디바이스 세팅
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

# Load Spread Sheet Dataset

In [6]:
# GOOGLE API KEY: json
key = files.upload()

Saving railway-tagtog-d24e6901ddb0.json to railway-tagtog-d24e6901ddb0.json


In [8]:
#로그인 위치
url = "https://tagtog.com/-login"

#다운로드 위치
file_url = 'https://www.tagtog.com/slayerzeroa/Railway_BERT/-downloads/dataset-as-anndoc' 
o_file = 'Railway_Annotation.zip'  
if os.path.exists(o_file):
    os.remove(o_file)

#로그인 정보
login_info = {
    'loginid' : 'slayerzeroa@gmail.com',
    'password' : 'gyysxw5rU3NzrYX',
}

#로그인
with requests.Session() as s:
    login_req = s.post(url, data=login_info)
    r = s.get(file_url)

    with open(o_file,"wb") as output:
        output.write(r.content)

In [10]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

#json key file 위치
json_file_name = './railway-tagtog-d24e6901ddb0.json'

# json key file을 이용하여 접속
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

#구글 스프레드 시트 주소
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1cpt2ad2pw0N2tC1H3MPtRAaZzFmgCh2If0gHOQKEIIo/edit#gid=0"

# 스프레드시트 문서 가져오기
doc = gc.open_by_url(spreadsheet_url)
## gc.create(spreadsheet_name) # 스프레드시트 생성


#시트 선택하기
sheet_name = "context"
worksheet = doc.worksheet(sheet_name) #해당 시트가 있는 경우 불러오기
## 403 error가 뜰 경우, google sheets API를 활성시켜줘야 함

# 시트의 모든 데이터 가져오기
values = worksheet.get_all_values()

# Pandas Dataframe화
header, rows = values[0], values[1:]
data = pd.DataFrame(rows, columns=header)
column_list = ['Doc', 'Start', 'Class', 'Part', 'Text','Label']
data.columns = column_list
print(data)

         Doc  Start Class  Part  \
0     200226   3874  e_22  s1p4   
1     200226   3874  e_22  s1p4   
2     200226    112   e_4  s1p5   
3     200226   3874  e_22  s1p4   
4     200226    112   e_4  s1p5   
...      ...    ...   ...   ...   
1388    mple  13368  e_23  s1p6   
1389    mple  13525  e_23  s1p6   
1390    mple  13608  e_23  s1p6   
1391    mple  13689  e_23  s1p6   
1392    mple  13763  e_23  s1p6   

                                                   Text  \
0                    5번째 차량(3513호) 앞 대차가 선로 우측으로 탈선하였다.   
1                    5번째 차량(3513호) 앞 대차가 선로 우측으로 탈선하였다.   
2     기관사가 정지신호를 무시하고 운행하다가 선로전 환기를 할출한 후 관제원의 되돌이 운...   
3                    5번째 차량(3513호) 앞 대차가 선로 우측으로 탈선하였다.   
4     기관사가 정지신호를 무시하고 운행하다가 선로전 환기를 할출한 후 관제원의 되돌이 운...   
...                                                 ...   
1388  화차 차축 베어링의 분해검수 주기와 교환 주기를 화차에 적재되는 화 물의 종류, 적...   
1389  컨테이너를 화차에서 들어 올릴 때 화차가 함께 들어 올려지다 떨어지지 않도록 작업 ...   
1390  화차의 발열 여부를 발견하기 위해 차축에 부착한 온도 테이프는 효과가 없는 

In [22]:
# Split labels based on whitespace and turn them into a list
labels = [i.split() for i in data['Label'].values.tolist()]
# Check how many labels are there in the dataset
unique_labels = set()

for lb in labels:
  [unique_labels.add(i) for i in lb if i not in unique_labels]
 
print(unique_labels)

# Map each label into its id representation and vice versa
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}
print(labels_to_ids)

print(labels)

{'O', 'OBJECT', 'ACCIDENT'}
{'ACCIDENT': 0, 'O': 1, 'OBJECT': 2}
[['O', 'OBJECT', 'O', 'O', 'O', 'O', 'OBJECT', 'O', 'O', 'ACCIDENT', 'O'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], 

In [13]:
example = data['Text'][0]
print(example)

label = data['Label'][0]
print(label)


5번째 차량(3513호) 앞 대차가 선로 우측으로 탈선하였다.
O OBJECT O O O O OBJECT O O ACCIDENT O


In [14]:
text_tokenized = tokenizer(example, padding='max_length', max_length=512, truncation=True, return_tensors="pt")
print(text_tokenized)

{'input_ids': tensor([[   101,    126,  48506,   9730,  44321,    113,  34487,  10884,  20309,
            114,   9531,   9069,  23466,  11287,   9428,  11261,   9604, 119281,
          11467,   9848,  18471,  12609,    119,    102,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              

In [15]:
print(tokenizer.decode(text_tokenized.input_ids[0]))

[CLS] 5번째 차량 ( 3513호 ) 앞 대차가 선로 우측으로 탈선하였다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [16]:
print(tokenizer.convert_ids_to_tokens(text_tokenized["input_ids"][0]))

['[CLS]', '5', '##번째', '차', '##량', '(', '351', '##3', '##호', ')', '앞', '대', '##차', '##가', '선', '##로', '우', '##측', '##으로', '탈', '##선', '##하였다', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]

In [17]:
word_ids = text_tokenized.word_ids()
print(tokenizer.convert_ids_to_tokens(text_tokenized["input_ids"][0]))
print(word_ids)

['[CLS]', '5', '##번째', '차', '##량', '(', '351', '##3', '##호', ')', '앞', '대', '##차', '##가', '선', '##로', '우', '##측', '##으로', '탈', '##선', '##하였다', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]

In [20]:
def align_label_example(tokenized_input, labels):

        word_ids = tokenized_input.word_ids()

        previous_word_idx = None
        label_ids = []
   
        for word_idx in word_ids:

            if word_idx is None:
                label_ids.append(-100)
                
            elif word_idx != previous_word_idx:
                try:
                  label_ids.append(labels_to_ids[labels[word_idx]])
                except:
                  label_ids.append(-100)
        
            else:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
      

        return label_ids

In [25]:
label = labels[0]
print(label)

['O', 'OBJECT', 'O', 'O', 'O', 'O', 'OBJECT', 'O', 'O', 'ACCIDENT', 'O']


In [26]:
label_all_tokens = True

new_label = align_label_example(text_tokenized, label)
print(new_label)
print(tokenizer.convert_ids_to_tokens(text_tokenized["input_ids"][0]))

label_all_tokens = False

new_label = align_label_example(text_tokenized, label)
print(new_label)
print(tokenizer.convert_ids_to_tokens(text_tokenized["input_ids"][0]))

[-100, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 0, 1, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

- 전처리 완료
- 학습을 위해 더 많은 데이터 구축... 필요

In [ ]:
import torch

def align_label(texts, labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids

class DataSequence(torch.utils.data.Dataset):

    def __init__(self, df):

        lb = [i.split() for i in df['labels'].values.tolist()]
        txt = df['text'].values.tolist()
        self.texts = [tokenizer(str(i),
                               padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for i in txt]
        self.labels = [align_label(i,j) for i,j in zip(txt, lb)]

    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels

In [ ]:
import numpy as np

df = df[0:1000]
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

In [ ]:
from transformers import BertForTokenClassification

class BertModel(torch.nn.Module):

    def __init__(self):

        super(BertModel, self).__init__()

        self.bert = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(unique_labels))

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output

In [ ]:
def train_loop(model, df_train, df_val):

    train_dataset = DataSequence(df_train)
    val_dataset = DataSequence(df_val)

    train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()

    best_acc = 0
    best_loss = 1000

    for epoch_num in range(EPOCHS):

        total_acc_train = 0
        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            loss, logits = model(input_id, mask, train_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][train_label[i] != -100]
              label_clean = train_label[i][train_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_train += acc
              total_loss_train += loss.item()

            loss.backward()
            optimizer.step()

        model.eval()

        total_acc_val = 0
        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].squeeze(1).to(device)
            input_id = val_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, val_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][val_label[i] != -100]
              label_clean = val_label[i][val_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_val += acc
              total_loss_val += loss.item()

        val_accuracy = total_acc_val / len(df_val)
        val_loss = total_loss_val / len(df_val)

        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(df_train): .3f} | Accuracy: {total_acc_train / len(df_train): .3f} | Val_Loss: {total_loss_val / len(df_val): .3f} | Accuracy: {total_acc_val / len(df_val): .3f}')

LEARNING_RATE = 5e-3
EPOCHS = 1
BATCH_SIZE = 2

model = BertModel()
train_loop(model, df_train, df_val)

In [ ]:
def evaluate(model, df_test):

    test_dataset = DataSequence(df_test)

    test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0.0

    for test_data, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_data['attention_mask'].squeeze(1).to(device)

            input_id = test_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, test_label)

            for i in range(logits.shape[0]):

              logits_clean = logits[i][test_label[i] != -100]
              label_clean = test_label[i][test_label[i] != -100]

              predictions = logits_clean.argmax(dim=1)
              acc = (predictions == label_clean).float().mean()
              total_acc_test += acc

    val_accuracy = total_acc_test / len(df_test)
    print(f'Test Accuracy: {total_acc_test / len(df_test): .3f}')


evaluate(model, df_test)

In [ ]:

def align_word_ids(texts):
  
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids


def evaluate_one_text(model, sentence):


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    print(sentence)
    print(prediction_label)

#Tokenizer
-> 목적에 알맞는 토크나이저로 튜닝

In [18]:
def tokenizing_(token_list, label_list, tokenizer):
  text_seq = []
  valid_seq = []
  label_seq = label_list
  for tokens, label in zip(token_list, label_list):
    new_target_text = []
    new_valid = []
    for i, (t, l) in enumerate(zip(tokens, label)):
      tokens_wordpiece = tokenizer.tokenize(t)
      new_v = [1] + [0]*(len(tokens_wordpiece)-1)
      new_target_text.extend(tokens_wordpiece)
      new_valid.extend(new_v)
    valid_seq.append(new_valid)
    text_seq.append(new_target_text)
  return text_seq, valid_seq, label_seq

In [19]:
token_list = sentences
label_list = list(data['Class'])

text_seq, valid_seq, label_seq = tokenizing_(token_list, label_list, tokenizer)

print(text_seq)
print(valid_seq)
print(label_seq)

print(len(text_seq[0]))
print(len(valid_seq[0]))


NameError: ignored

- 개체명 사이의 관계 추출을 할 때는 입력값으로 (문장, 관계를 보고자하는 개체1,
관계를 보고자하는 개체 2) 형태로 문장과 개체 모두가 주어져야 함
- 따라서 모든 단어 쌍에 대해 구분을 위해 특수 token을 부여하여 이를 구별함

- 전체 텍스트에서 문장 분할
- 관계가 있는 두 문장을 left, right로 나누고
- 전체 텍스트에서 해당 하는 문장 있으면 특수 토큰 추가
-> 정규표현식 사용 re.sub

In [ ]:
# 관계있는 Dataset Tokenizing?
# 특수 토큰 집어넣는 일
def tokenizing_rel(token_list, label_list, rel_list, tokenizer):
  text_seq = []
  valid_seq = []
  final_left_seq = []
  final_right_seq = []
  label_seq = []

  for tokens, label, rel in zip(token_list, label_list, rel_list):
    lefts, rights, labels = rel

    for le, ri, la in zip(lefts, rights, labels):
      new_target_text = []
      new_valid = []
      new_left = []
      new_right = []
      before_label = -1

      for i, (t, l) in enumerate(zip(tokens, label)):
        tokens_wordpiece = tokenizer.tokenize(t)
        if before_label != l and l != -1:
          if l == le:
            tokens_wordpiece = ["*"] + tokens_wordpiece
          elif l == ri:
            tokens_wordpiece = ["#"] + tokens_wordpiece
        
        if l != -1 and i+1 != len(label) and l != label[i+1]:
          if l == le:
            tokens_wordpiece = tokens_wordpiece + ["*"]
          elif l == ri:
            tokens_wordpiece = tokens_wordpiece + ["#"]
        
        if l != -1 and i+1 == len(label):
          if l == le:
            tokens_wordpiece = tokens_wordpiece + ["*"]
          elif l == ri:
            tokens_wordpiece = tokens_wordpiece + ["#"]
        
        before_label = l
        new_v = [l]*(len(tokens_wordpiece))
        new_target_text.extend(tokens_wordpiece)
        new_valid.extend(new_v)
      
      leftaa = [float(x == le) for x in new_valid]
      rightaa = [float(x == ri) for x in new_valid]
      final_left_seq.append(leftaa)
      final_right_seq.append(rightaa)
      text_seq.append(new_target_text)
      label_seq.append(la)
    return text_seq, final_left_seq, final_right_seq, label_seq

In [ ]:
def make_greedy_label(rel_list, dic_rel_label, max_labels):
  final_rel_list = []

  for rel, max_lab in zip(rel_list, max_labels):
    temp_les = []
    temp_re =[]
    rel_labels = rel[4]
    rel_lefts = rel[5]
    rel_rights = rel[6]
    max_value = max_lab
    l, r = make_fair_by_max(max_value)
    new_l = []
    new_r = []
    qomax = len(rel_labels)
    z = 0

    for ll, rr in zip(l, r):
      q = 0
      laba = 0
      for lf, ri, rel_label in zip(rel_lefts, rel_rights, rel_labels):
        if ll == lf and rr == ri:
          q = 1
          laba = dic_rel_label[rel_label]
      
      if q == 1:
        new_l.append(ll)
        new_r.append(rr)
        temp_les.append(laba)
      elif z < 1.5*qomax:
        new_l.append(ll)
        new_r.append(rr)
        z = z + 1
        temp_les.append(0)
      
    final_rel_list.append((new_l, new_r, temp_les))
  return final_rel_list

#BERT Model Fine Tuning by MLM

+MLM학습을 위한 Plain Text 받아오기

In [ ]:
#압축파일 풀기
folder_path = "./tagtog_relation_extraction/"

zip_ = zipfile.ZipFile("Railway_Annotation.zip")
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
zip_.extractall(folder_path)

#폴더 경로
folder_name = "./tagtog_relation_extraction/Railway_BERT/"

#context list   # context == plain.html
context_name_list = os.listdir(folder_name + "plain.html/pool")
print(context_name_list)

#relation 폴더 경로   # relation == ann.json
relation_folder_paths = glob.glob(folder_name + "ann.json/master/pool")

#context 폴더 경로
# contexts_folders_paths = glob.glob(folder_name + "plain.html/pool/*")
contexts_folders_paths = [folder_name + "plain.html/pool/" + c for c in context_name_list]

#anntation_lenged 정보
annotations_legend = folder_name + "annotations-legend.json"
with open(annotations_legend,"r") as f:
    annotations_legend = json.load(f)

['a3DnzcVSvJKG25OeznRpiW5zLgi8-sample.pdf.plain.html', 'awUyBdL4lPpVlBOPjSzMhAuRk81a-20181208.pdf.plain.html', 'arqduk0rnomRe20HKEeXlVPK8qx8-20191022.pdf.plain.html', 'aVuARZFRfgJMD0E67Coa.7G8Z9Pe-20200226.pdf.plain.html', 'aPSRwvP5rtizz2O093I3oFX9uzjS-20180818.pdf.plain.html']


In [ ]:
def remove_tag(html_text):
  check = html_text
  check = re.sub('(<([^>]+)>)', '', check).strip()
  check = re.sub('\n', '', check)
  check = re.sub(' +', ' ', check)
  check = check[41:]
  check = re.sub('·','', check)
  check = re.sub('○', '', check)
  return check

In [ ]:
check = ''

for context_path in contexts_folders_paths:
  with open(context_path, "r") as f:
    sample = f.read()
    sample = remove_tag(sample)
  print(sample)
  check += sample

check = sent_tokenize(check)

고서 번호: ARAIB/R 2020 - 3 철도사고 조사보고서 한국철도공사 경전선(완사역→진주역 사이, 삼랑진역 기점 103.286km) 제3081화물열차(디젤 7620호 + 화차 25량) 열차탈선 2019년 2월 18일(월) 21시 56분경 2020. 6. 19. 항공･철도사고조사위원회 이 조사보고서는 「항공․철도사고조사에 관한 법률」제 2조에 의거 사고조사가 이루어졌으며, 제25조에 따라 작 성되었다. 같은 법률 제1조에서 「철도사고 조사는 독립적이고 공정 한 조사를 통하여 사고 원인을 정확하게 규명함으로써 철도 사고의 예방과 안전 확보에 이바지함」을 목적으로 하고 있 다. 또한, 제30조에 따라 사고조사는 민․형사상 책임과 관련된 사 법절차, 행정처분 절차 또는 행정 쟁송 절차와 분리․수행되어 야 하고, 제32조에서위원회에 진술․증언․자료 등의 제출 또는 답변을 한 사람은 이를 이유로 해고․전보․징계․부당한 대우 또 는 그 밖에 신분이나 처우와 관련하여 불이익을 받지 아니 한다.라고 규정하고 있다. 그러므로 이 조사보고서는 철도 분야의 안전을 증진시킬 목 적 이외의 용도로 사용되어서는 아니 된다. 차례 한국철도공사 경전선 완사역∼진주역간 화물열차 탈선사고 조사보고서 - i - 차 례 한국철도공사 경전선 완사역∼진주역간 화물열차 탈선사고  1 개요  2 1. 사실 정보  3 1.1 사고의 경위  3 1.2 피해 사항  4 1.3 인적 정보 및 업무 수행사항  4 1.4 현장 정보  10 1.5 열차와 차량 정보  11 1.6 차량정비단의 차축 베어링 분해 검수 실태  16 1.7 신호 및 전차선 정보  18 1.8 기상 정보  19 2. 분석  20 2.1 업무 수행사항 분석  20 2.2 차축 베어링 분해 검사  22 2.3 컨테이너 상하차 작업 분석  25 2.4 그리스 성분 분석  26 2.5 종합 분석  29 3. 결론  30 3.1 조사 결과  30 3.2 사고 원인  32 4. 안전 권고  33 4.1 한국철도공사에 대하여  33 제목 한국철

In [ ]:
print(len(check))

1392


In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

MLM Dataset = Fine Tuning 하기 위한 데이터 셋 생성

In [ ]:
# Dataset: plain.html
class MLMTrainDataset_Sik(Dataset):
  
  def __init__(self, max_len, mask_prob, mask_token, sep_token, cls_token, all_sents,
               tokenizer, pad_token):
    self.all_sents = all_sents
    self.num_vocabs = tokenizer.vocab_size
    self.max_len = max_len
    self.mask_prob = mask_prob
    self.mask_token = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(mask_token))
    self.cls_token = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(cls_token))
    self.sep_token = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sep_token))
    self.tokenizer = tokenizer
    self.pad_token = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(pad_token))
  
  def __len__(self):
    return len(self.all_sents)
  
  def __getitem__(self, index):
    sents = self.all_sents[index]
    sents = self.tokenizer.convert_tokens_to_ids(self.tokenizer.tokenize(sents))
    tokens = []
    sents = np.array(sents)
    inputs = sents
    labels = inputs.copy()

    probability_matrix = torch.full(labels.shape, self.mask_prob)

    masked_indices = torch.bernoulli(probability_matrix).bool()
    
    labels[~masked_indices] = -100

    indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices

    inputs[indices_replaced] = self.tokenizer.convert_tokens_to_ids(self.mask_token)

    indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced

    random_words = torch.randint(len(self.tokenizer), labels.shape, dtype = torch.long)

    inputs[indices_random] = random_words[indices_random]
    inputs = list(inputs)
    labels = list(labels)
    valid_length = min(len(inputs), self.max_len-2)
    tokens = self.cls_token + inputs[:self.max_len-2] + self.sep_token
    labels = [-100] + labels[:self.max_len-2] + [-100]

    mask_len = self.max_len - len(tokens)

    tokens = tokens + self.pad_token * mask_len
    labels = labels + [-100] * mask_len
    attention_masks = [0.0]*len(tokens)
    for i in range(valid_length):
      attention_masks[i] = 1.0
    
    return torch.LongTensor(tokens), torch.LongTensor(labels), valid_length, torch.LongTensor(attention_masks)

In [ ]:
# set token
sep_token="[SEP]"
pad_token="[PAD]"
cls_token="[CLS]"
mask_token="[MASK]"
max_len = 60
mask_prob = 0.15

# make instance
test_instance = MLMTrainDataset_Sik(max_len, mask_prob, mask_token, sep_token, cls_token, check, bert_tokenizer, pad_token)

In [ ]:
# check instance
print(test_instance[3])

(tensor([   101,   9959,  28000,    100,  87624,  12945,    100,  20626,  12945,
         68673,   9638,   9678,  12945,  30005,    100,  37321,   1890,   9959,
           100,   1730,  87624,  12945,  11664,  20626,  86580,  42300,   9341,
           100,    100,   9672,    123,  20626,  10530,    100,  41521,   9405,
         11664,    100,  49636,    100,    100,  12965, 119210,  24098,    117,
          9672,  69168,    100,  10530,  22799,   9652,   9434,  13628,    119,
           102,      0,      0,      0,      0,      0]), tensor([ -100,  -100,  -100, 10101,  -100,  -100, 11664,  -100,  -100,  -100,
         -100,  -100,  -100,  -100, 11664,  -100,  -100,  -100, 28000,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100, 88350,  1891,  -100,
         -100,  -100,  -100,  9637,  -100,  -100,  -100, 20626,  -100,  9638,
        35866,  -100,  -100,  -100,  -100,  -100,  -100, 20626,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100, 

<ipython-input-19-6b757058d52f>:56: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.LongTensor(tokens), torch.LongTensor(labels), valid_length, torch.LongTensor(attention_masks)


#BERT Fine-Tuning

#BERT Entity Extration

In [ ]:
# 개체명 인식 모델 Class 코드
class Bert_entity(nn.Module):
  def __init__(self, model, num_labels):
    super().__init__()
    self.model = model
    self.hidden_size = 768
    self.block_size = 64
    self.hidden_layer = nn.Linear(768, 768)
    self.classifier = nn.Linear(768, num_labels)
    self.dropout = nn.Dropout(0.4)
    self.num_labels = num_labels
    self.crf = CRF(num_tags = num_labels, batch_first=True)
  
  def forward(self,
              input_ids = None,
              attention_mask = None,
              labels = None,
              valid_ids = None,
              label_mask = None):
    
    output = self.model(input_ids = input_ids, attention_mask = attention_mask)
    last_hidden_state = output.last_hidden_state
    valid_output = torch.zeros(last_hidden_state.size()[0], last_hidden_state.size()[1],
                               last_hidden_state.size()[2], dtype=torch.float32, device='cuda:1')
    valid_masks = torch.zeros(last_hidden_state.size()[0], last_hidden_state.size()[1],
                              dtype=torch.bool, device='cuda:1')
    
    for i in range(last_hidden_state.size()[0]):
      jj = -1
      for j in range(last_hidden_state.size()[1]):
        if valid_ids[i][j].item() == 1:
          jj += 1
          valid_masks[i][jj] = 1
          valid_output[i][jj] = last_hidden_state[i][j]
      
      relations = self.dropout(valid_output)
      logits = self.classifier(relations)
      loss = None
      if labels is not None:
        if attention_mask is not None:
          loss, logits = self.crf(logits, labels, mask = valid_masks, reduction = 'token_mean'), self.crf.decode(logits, mask=valid_masks)
        else:
          loss, logits = self.crf(logits, labels, mask = valid_masks, reduction = 'token_mean'), self.crf.decode(logits, mask=valid_masks)
      return -loss, logits, last_hidden_state, valid_output, output.attentions


In [ ]:
learning_rate = 0.001
weight_decay = 0.01

model = BertModel.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model = Bert_entity(model, 4)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
labels_list = ['[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [ ]:
loss_ = [] # loss 리스트
accuracy_list = []    # accuracy 리스트
trainloader = torch.utils.data.DataLoader(test_instance, shuffle=True)
num_epochs = 100

for epochs in range(num_epochs):
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    inputs, values, valid_len, attention = data
    optimizer.zero_grad() # Optimizer 초기화

    outputs = model(inputs, attention)             # 예측값 산출
    loss = criterion(outputs, values) # Loss 계산
    loss.backward()                          # 역전파
    optimizer.step()                            # 가중치 업데이트

    running_loss += loss.item()          # Epoch 평균 Loss를 구하기 위해 더하기
  print(f'epoch[{epoch+1}/{num_epoch}]', f'Loss : {running_loss}')

<ipython-input-19-6b757058d52f>:56: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.LongTensor(tokens), torch.LongTensor(labels), valid_length, torch.LongTensor(attention_masks)


RuntimeError: ignored

# BERT Relation

In [ ]:
# 개체간 관계 추출 모델 Class 코드
class Bert_relation(nn.Module):
  def __init__(self, model, num_labels):
    super().__init__()
    self.model = model
    self.hidden_size =768
    self.block_size = 64

    self.head_extractor = nn.Linear(768, 768)
    self.tail_extractor = nn.Linear(768, 768)

    self.bilinear_classifier = nn.Linear(768 * 3, num_labels)

    self.dropout = nn.Dropout(0.1)
    self.num_labels = num_labels
  
  def get_hrts(self, valid_output, left_ids, right_ids):
    left_masks = left_ids.unsqueeze(-1).expand(valid_output.size()).float()
    right_masks = right_ids.unsqueeze(-1).expand(valid_output.size()).float()

    left_rep = torch.logsumexp(valid_output * left_masks, dim=1)
    right_rep = torch.logsumexp(valid_output * right_masks, dim=1)

    return left_rep, right_rep
  
  def forward(self,
              input_ids=None,
              attention_mask=None,
              labels=None,
              valid_ids=None,
              label_mask=None,
              left_ids=None,
              right_ids=None):
    output = self.model(input_ids=input_ids, attention_mask=attention_mask)
    last_hidden_state = output.last_hidden_state
    pooled_output = output.pooler_output
    context = self.dropout(pooled_output)
    valid_output = last_hidden_state

    left_rep, right_rep = self.get_hrts(valid_output, left_ids, right_ids)

    left_rep = self.head_extractor(left_rep)
    right_rep = self.head_extractor(right_rep)

    bl = torch.cat([context, left_rep, right_rep], dim=-1)

    logits = self.bilinear_classifier(bl)
    loss = None

    if labels is None:
      loss_fct = torch.nn.BCEWithLogitsLoss()
      loss = loss_fct(logits, labels.float())

    output = logits

    return loss, output if loss is not None else output

In [ ]:
def find_most_similar_words(word, total_words):
  q = []

  for a in total_words:
    k = jaccard_similarity(word, a[0])
    q.append((a, k))
  q.sort(key=lambda x:x[1])
  qq = q[-20:]
  new_q = []
  for qa in qq:
    if qa[1] > 0.05:
      new_q.append(qa)
  
  return new_q

In [ ]:
def jaccard_similarity(word1, word2):
  word1 = word1.lower()
  word2 = word2.lower()
  trigram1 = trigram_from_word(word1)
  trigram2 = trigram_from_word(word2)

  set_trigram = set(trigram1)
  set_trigram = set_trigram.union(set(trigram2))
  con_trigram = [t in trigram2 for t in trigram1]
  qq = sum(con_trigram)
  la = 0.0
  la = float(float(qq)/float(len(set_trigram)+1e-9))

  return la

In [ ]:
def trigram_from_word(word):
  if len(word) < 3:
    return [word]
  
  else:
    trigram = [word[i:i+3] for i in range(len(word)-3)]
    return trigram

In [ ]:
str_ = output_string.getvalue()
new_str = re.sub("\n", "", str_)
new_str = re.sub("·", "", new_str)

In [ ]:
word_tokens = word_tokenize(new_str)
sent_tokens = sent_tokenize(new_str)

In [ ]:
okt = Okt()
noun = okt.nouns(text)
for i, v in enumerate(noun):
  if len(v) < 2:
    noun.pop(i)

count = Counter(noun)

In [ ]:
# 단어 빈도 카운트
noun_list = count.most_common(100)
for v in noun_list:
  print(v)

#BERT Training

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')

In [ ]:
inputs = bert_tokenizer(check, return_tensors='pt')

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
rand = torch.rand(inputs.input_ids.shape)

mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) # 101, 102번 토큰 제외하고 15% 위치 선별

### padding이 포함된 경우(0번도 추가 제외) ###
# mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)

selection = torch.flatten((mask_arr[0]).nonzero())

In [ ]:
import numpy as np

selection_val = np.random.random(len(selection)) # selection의 위치마다 0~1 값 부여

mask_selection = selection[np.where(selection_val >= 0.2)[0]] # 80% : Mask 토큰 대체
random_selection = selection[np.where(selection_val < 0.1)[0]] # 10% : 랜덤 토큰 대체

print(random_selection) # tensor([ 30,  95, 143])
print(mask_selection)

In [ ]:
inputs.input_ids[0, mask_selection] = 103
inputs.input_ids[0, random_selection] = torch.randint(0, 30522, size = random_selection.shape)

print(inputs)

In [ ]:
outputs = model(**inputs)